# Spettrometro


## TO DO:
1) capire bene quali lambda usare

In [150]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import chi2
from iminuit import Minuit
from iminuit.cost import LeastSquares
from uncertainties import ufloat
import uncertainties.umath as uu

In [151]:
f_mul = 4 #fattore di correzione (non possiamo usare la sensibilita di 1')

### Angolo al vertice

In [152]:
theta_i = np.radians(np.array([52 + 40/60, 54 + 41/60, 56 + 30/60]))
theta_f = np.radians(np.array([172 + 29/60, 174 + 35/60, 176 + 15/60]))
alpha_val_array = theta_f - theta_i
alpha_val = np.mean(alpha_val_array) / 2
alpha_err = ( np.sqrt(2) * np.radians(1/60 * f_mul) ) / np.sqrt(3) #errore della media
alpha = ufloat(alpha_val, alpha_err)

### Legge di Cauchy (mercurio)

In [153]:
theta_i_hg = ufloat(np.radians(30),np.radians(1/60 * f_mul))
theta_f_hg_senzaerr = np.radians(np.array([77 + 34/60, 77 + 50/60, 78 + 2/60, 78 + 10/60])) #mercurio
theta_f_hg = []

for i in range(len(theta_f_hg_senzaerr)):
    theta_f_hg.append(ufloat(theta_f_hg_senzaerr[i],np.radians(1/60 * f_mul)))

theta_f_hg = np.array(theta_f_hg)

delta_hg = theta_f_hg - theta_i_hg

lambda_hg = np.array([579, 546.1, 502.5, 435.8]) * 10 ** (-9) #da confermare

In [154]:
n = []
n_val = []
n_err = []

for i in range(len(delta_hg)):
    n.append( uu.sin((delta_hg[i] + alpha)/2) / uu.sin(alpha/2) )
    n_val.append(n[i].nominal_value)
    n_err.append(n[i].std_dev)

In [155]:
def model(x,a,b): #legge di Cauchy
    return a + b/(x**2)

minimi_quadrati = LeastSquares(lambda_hg, n_val, n_err, model)

m = Minuit(minimi_quadrati, a=1, b=0)

m.migrad()
m.hesse()
display(m)

def func(x):
    return chi2.cdf(x,m.ndof)

pvalue = func(np.inf) - func(m.fval)
print("p value: " + str(pvalue))

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.407 (χ²/ndof = 1.2)      │              Nfcn = 51               │
│ EDM = 3.5e-08 (Goal: 0.0002)     │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ a    │  1.6088   │  0.0028   │            │            │         │         │       │
│ 1 │ b    │  2.4e-15  │  0.7e-15  │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────────────────────────────┐
│   │                    a                    b │
├───┼───────────────────────────────────────────┤
│ a │             8.14e-06 -1.9927266089092e-18 │
│ b │ -1.9927266089092e-18             5.12e-31 │
└───┴───────────────────────────────────────────┘

p value: 0.3000752382155041


In [156]:
print("a: " + str(m.values[0]) + "    err: " + str(m.errors[0]))
print("b: " + str(m.values[1] * 10 **(12)) + " micrometri^2" + "    err: " + str(m.errors[1]*10**(12)))

a: 1.6087514636456364    err: 0.0028526220992847375
b: 0.0024454775756997124 micrometri^2    err: 0.0007157000870714331


### Idrogeno (non esce, ma non serve)

In [157]:
f_mul2 = 4 #fattore che moltiplica f_mul per far uscire il fit con l'idrogeno

In [158]:
theta_i_h = ufloat(np.radians(34 + 45/60), np.radians(1/60 * f_mul * f_mul2))
theta_f_h_senzaerr = np.radians(np.array([80 + 55/60, 84, 84 + 16/60])) #idrogeno (ignoto)
theta_f_h = []

for i in range(len(theta_f_h_senzaerr)):
    theta_f_h.append(ufloat(theta_f_h_senzaerr[i],np.radians(1/60 * f_mul * f_mul2)))

theta_f_h = np.array(theta_f_h)

delta_h = theta_f_h - theta_i_h

lambda_h = np.array([667.8, 492.1, 447.1]) * 10 ** (-9) #da confermare

n = []
n_val = []
n_err = []

for i in range(len(delta_h)):
    n.append( uu.sin((delta_h[i] + alpha)/2) / uu.sin(alpha/2) )
    n_val.append(n[i].nominal_value)
    n_err.append(n[i].std_dev)

minimi_quadrati = LeastSquares(lambda_h, n_val, n_err, model)

m = Minuit(minimi_quadrati, a=1, b=0)

m.migrad()
m.hesse()
display(m)

def func(x):
    return chi2.cdf(x,m.ndof)

pvalue = func(np.inf) - func(m.fval)
print("p value: " + str(pvalue))

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.825 (χ²/ndof = 2.8)      │              Nfcn = 49               │
│ EDM = 1.99e-09 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ a    │   1.573   │   0.008   │            │            │         │         │       │
│ 1 │ b    │ 13.2e-15  │  2.0e-15  │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────────────────────────────┐
│   │                    a                    b │
├───┼───────────────────────────────────────────┤
│ a │             6.34e-05 -15.204789169755e-18 │
│ b │ -15.204789169755e-18             3.97e-30 │
└───┴───────────────────────────────────────────┘

p value: 0.09280317148551176
